In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D, Conv3D
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils, generic_utils

import tensorflow as tf
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pyplot as pyplot

import numpy as np
import scipy.io
import cv2
from PIL import Image
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn import preprocessing

img_rows, img_cols, img_depth = 64, 64, 64

/home/xd/ws/tf/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/xd/ws/tf/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
root_dir = '/home/xd/Documents/dataset/'
listing = os.listdir(root_dir)
maxval = 0.46
classes = {'left':0, 'right':1, 'click':2, 'wrist':3}

In [3]:
frames = []
labels = []
for file in listing:
    file = root_dir + file
    data = scipy.io.loadmat(file)
    images = data['images']
    label = classes[data['ground_truth'][0]]
    num_frame = images.shape[0]
    
    if num_frame > 500:
        continue
    for i in range(50, 150, 32):
        vol = images[i:i+64]
        vol = vol.reshape(64,64,64)
        frames.append(vol)
        labels.append(label)

In [4]:
num_samples = len(frames)
frames_arr =np.array(frames)

In [5]:
X = np.zeros((num_samples, 1, img_rows, img_cols, img_depth))
for h in range(num_samples):
    X[h][0][:][:][:] = frames_arr[h,:,:,:]
    
X = np.swapaxes(X, 1, 4)
print(X.shape, 'train samples')

(1108, 64, 64, 64, 1) train samples


In [6]:
labels_arr = np.array(labels) - 1

In [8]:
batch_size = 4
nb_classes = 4
nb_epoch = 100
patch_size = 64

In [15]:
y = np_utils.to_categorical(labels_arr, nb_classes)

nb_filters = [32, 32]
nb_pool = [2, 2]
nb_conv = [5, 5]

X = X.astype('float32')
X -= np.mean(X)
X /= np.max(X)

In [16]:
model = Sequential()
print(nb_filters[0], 'filters')
print('input shape', img_rows, 'rows', img_cols, 'cols', patch_size, 'patchsize')

model.add(Conv3D(32,
                 (5, # depth
                  5, # rows
                  5), # cols
                 input_shape=(patch_size, img_rows, img_cols, 1),
                 activation='relu',
                 padding='SAME',
))
model.add(MaxPooling3D(pool_size=(nb_pool[0], nb_pool[0], nb_pool[0])))
model.add(Dropout(0.5))
# model.add(Conv3D(32,
#                  (3, # depth
#                   3, # rows
#                   3), # cols
#                  activation='relu',
#                  padding='SAME',
# ))
# model.add(MaxPooling3D(pool_size=(nb_pool[0], nb_pool[0], nb_pool[0])))
# model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(84, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, kernel_initializer='normal'))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics=['mse', 'accuracy'])

32 filters
input shape 64 rows 64 cols 64 patchsize


In [17]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

hist = model.fit(
X_train,
y_train,
validation_data=(X_val, y_val),
batch_size=batch_size,
epochs=nb_epoch,
shuffle=True
)

Train on 886 samples, validate on 222 samples
Epoch 1/100


ResourceExhaustedError: OOM when allocating tensor of shape [1048576,84] and type float
	 [[Node: training/RMSprop/zeros_2 = Const[dtype=DT_FLOAT, value=Tensor<type: float shape: [1048576,84] values: [0 0 0]...>, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'training/RMSprop/zeros_2', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/xd/ws/tf/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/xd/ws/tf/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/xd/ws/tf/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 486, in start
    self.io_loop.start()
  File "/home/xd/ws/tf/lib/python3.5/site-packages/tornado/platform/asyncio.py", line 127, in start
    self.asy

In [ ]:
train_loss = hist.history['loss']
val_loss = hist.history['val_loss']
train_acc = hist.history['acc']
val_acc = hist.history['val_acc']

In [ ]:
plt.plot(train_loss); plt.title('train loss')
plt.plot(val_loss); plt.title('val loss')
plt.legend(['train loss', 'val_loss'])

In [ ]:
plt.plot(train_acc); plt.title('train acc')
plt.plot(val_acc); plt.title('val acc')
plt.legend(['train loss', 'val loss'])
plt.grid(True)
plt.ylim(0,1)

In [12]:
X.shape

(1108, 64, 64, 64, 1)

In [13]:
from keras import backend as K
K.clear_session()

In [18]:
np.save('X.npy', X)

In [19]:
np.save('y.npy', y)